In [1]:
import pandas as pd

In [176]:
def wrangle(path):
    df = pd.read_csv(path)

    df['Salary'] = df['Salary Estimate'].str.split(' ', expand = True)[0]
    df['Salary'] = df['Salary'].str.replace('$','').str.replace('K','')
    df['min_salary'] = df['Salary'].str.split('-', expand = True)[0]
    df['max_salary'] = df['Salary'].str.split('-', expand = True)[1]

    col_drop = ["index", "Salary Estimate"]
    df["Company Name"] = df["Company Name"].str.split("\n", expand = True)[0]

    df_split = df['Location'].str.split(', ', expand=True)

    # Create 'job state' column based on the length condition for the second column
    df['job state'] = df_split.apply(lambda row: row[2] if len(str(row[1])) != 2 else row[1], axis=1)

    df["Job Description"] = df["Job Description"].str.replace("\n", ' ')
    

    df.drop(columns = col_drop, inplace = True)


    return df
    
df = wrangle("Uncleaned_DS_jobs.csv")
df.head(2)

,Job Title,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Salary,min_salary,max_salary,job state
0,Sr Data Scientist,Description The Senior Data Scientist is resp...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137-171,137,171,NY
1,Data Scientist,"Secure our Nation, Ignite your Future Join th...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,137-171,137,171,VA


In [ ]:
def wrangle(path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(path)

    # Extract only the salary value (before any space) from the 'Salary Estimate' column
    # This is because the salary is the first part of the 'Salary Estimate' column
    df['Salary'] = df['Salary Estimate'].str.split(' ', expand=True)[0]

    # Remove the dollar sign and the 'K' (for thousands) from the salary values for easier processing
    # $ and K symbols are typically used in salary estimates, so they are being removed
    df['Salary'] = df['Salary'].str.replace('$', '').str.replace('K', '')

    # Split the salary into two parts: 'min_salary' and 'max_salary'
    # The 'min_salary' takes the first part of the salary (before the dash)
    df['min_salary'] = df['Salary'].str.split('-', expand=True)[0]

    # The 'max_salary' takes the second part of the salary (after the dash)
    df['max_salary'] = df['Salary'].str.split('-', expand=True)[1]

    # Drop unnecessary columns, in this case, 'index' and 'Salary Estimate'
    # 'index' is redundant and 'Salary Estimate' is already split into a more useful column
    col_drop = ["index", "Salary Estimate"]

    # Clean the 'Company Name' column by removing any newline characters
    # The company name might have extra information after a newline, so we take only the first part
    df["Company Name"] = df["Company Name"].str.split("\n", expand=True)[0]

    # Split the 'Location' column into two parts: city and state (assuming it's in the format 'City, State')
    df_split = df['Location'].str.split(', ', expand=True)

    # Create a new column 'job state' based on the second column of the split 'Location'
    # If the length of the second column is not 2 (state abbreviations are usually 2 characters), 
    # it takes the value from the third column instead
    df['job state'] = df_split.apply(lambda row: row[2] if len(str(row[1])) != 2 else row[1], axis=1)

    # Replace newline characters in the 'Job Description' column with spaces to clean up the text
    df["Job Description"] = df["Job Description"].str.replace("\n", ' ')

    # Drop the columns specified earlier ('index' and 'Salary Estimate') as they are no longer needed
    df.drop(columns=col_drop, inplace=True)

    # Return the cleaned DataFrame
    return df

# Example usage: load and clean the dataset
df = wrangle("Uncleaned_DS_jobs.csv")

# Display the first two rows of the cleaned DataFrame to check the output
df.head(2)


In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Job Description    672 non-null    object 
 2   Rating             672 non-null    float64
 3   Company Name       672 non-null    object 
 4   Location           672 non-null    object 
 5   Headquarters       672 non-null    object 
 6   Size               672 non-null    object 
 7   Founded            672 non-null    int64  
 8   Type of ownership  672 non-null    object 
 9   Industry           672 non-null    object 
 10  Sector             672 non-null    object 
 11  Revenue            672 non-null    object 
 12  Competitors        672 non-null    object 
 13  Salary             672 non-null    object 
 14  min_salary         672 non-null    object 
 15  max_salary         672 non-null    object 
 16  job state          649 non

In [64]:
df["Rating"].unique()

array([ 3.1,  4.2,  3.8,  3.5,  2.9,  3.9,  4.4,  3.6,  4.5,  4.7,  3.7,
        3.4,  4.1,  3.2,  4.3,  2.8,  5. ,  4.8,  3.3,  2.7,  2.2,  2.6,
        4. ,  2.5,  4.9,  2.4, -1. ,  2.3,  4.6,  3. ,  2.1,  2. ])

In [66]:
df["Rating"][(df["Rating"].astype("string") != df["Company Name"].str.split("\n", expand = True)[1])]

167    4.2
179    3.6
308    5.0
Name: Rating, dtype: float64

In [73]:
(df["Company Name"].str.split("\n", expand = True)[0].str.contains("'\'")).unique()

array([False])

In [59]:
df["Company Name"].loc[167]

'Autodesk\n4.0'

In [76]:
df['Location'].unique()

array(['New York, NY', 'Chantilly, VA', 'Boston, MA', 'Newton, MA',
       'Santa Barbara, CA', 'Cambridge, MA', 'Bedford, MA',
       'San Diego, CA', 'Chicago, IL', 'Herndon, VA', 'Saint Louis, MO',
       'Richland, WA', 'Northbrook, IL', 'Washington, DC', 'Remote',
       'Memphis, TN', 'Plano, TX', 'West Grove, PA', 'Phoenix, AZ',
       'Appleton, WI', 'Atlanta, GA', 'Orlando, FL', 'Lexington, MA',
       'McLean, VA', 'San Francisco, CA', 'Sheboygan, WI',
       'United States', 'Bothell, WA', 'Lincoln, NE', 'Overland Park, KS',
       'Santa Monica, CA', 'Portsmouth, NH', 'Ewing, NJ',
       'South San Francisco, CA', 'Palo Alto, CA', 'Bellevue, WA',
       'New Orleans, LA', 'Akron, OH', 'Fort Wayne, IN', 'Woburn, MA',
       'Carson, CA', 'Coral Gables, FL', 'Santa Clara, CA',
       'Brisbane, CA', 'Winter Park, FL', 'Redwood City, CA',
       'Peoria, IL', 'Ipswich, MA', 'Carmel, IN', 'Emeryville, CA',
       'Gaithersburg, MD', 'Longmont, CO', 'Austin, TX', 'Yakima, WA',
 

In [81]:
#df['Headquarters'].unique()
contains_special_chars = df['Headquarters'].str.contains(r'[^a-zA-Z0-9]', regex=True)
(contains_special_chars == False).unique()

array([False])

In [83]:
df['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', '-1', 'Unknown'],
      dtype=object)

In [84]:
df['Founded'].unique()

array([1993, 1968, 1981, 2000, 1998, 2010, 1996, 1990, 1983, 2014, 2012,
       2016, 1965, 1973, 1986, 1997, 2015, 1945, 1988, 2017, 2011, 1967,
       1860, 1992, 2003, 1951, 2005, 2019, 1925, 2008, 1999, 1978, 1966,
       1912, 1958, 2013, 1849, 1781, 1926, 2006, 1994, 1863, 1995,   -1,
       1982, 1974, 2001, 1985, 1913, 1971, 1911, 2009, 1959, 2007, 1939,
       2002, 1961, 1963, 1969, 1946, 1957, 1953, 1948, 1850, 1851, 2004,
       1976, 1918, 1954, 1947, 1955, 2018, 1937, 1917, 1935, 1929, 1820,
       1952, 1932, 1894, 1960, 1788, 1830, 1984, 1933, 1880, 1887, 1970,
       1942, 1980, 1989, 1908, 1853, 1875, 1914, 1898, 1956, 1977, 1987,
       1896, 1972, 1949, 1962])

In [85]:
df['Type of ownership'].unique()

array(['Nonprofit Organization', 'Company - Public',
       'Private Practice / Firm', 'Company - Private', 'Government',
       'Subsidiary or Business Segment', 'Other Organization', '-1',
       'Unknown', 'Hospital', 'Self-employed', 'College / University',
       'Contract'], dtype=object)

In [86]:
 # 9   Industry           672 non-null    object 
 # 10  Sector             672 non-null    object 
 # 11  Revenue            672 non-null    object 
 # 12  Competitors

In [89]:
df['Revenue'].unique()

array(['Unknown / Non-Applicable', '$1 to $2 billion (USD)',
       '$100 to $500 million (USD)', '$10+ billion (USD)',
       '$2 to $5 billion (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 billion (USD)', '$10 to $25 million (USD)',
       '$25 to $50 million (USD)', '$50 to $100 million (USD)',
       '$1 to $5 million (USD)', '$5 to $10 million (USD)',
       'Less than $1 million (USD)', '-1'], dtype=object)

In [97]:
min_salary = df['Salary'].str.split("-", expand = True)[0]
max_salary = df['Salary'].str.split("-", expand = True)[1]

In [116]:
df['Location']

0           New York, NY
1          Chantilly, VA
2             Boston, MA
3             Newton, MA
4           New York, NY
             ...        
667         Fort Lee, NJ
668    San Francisco, CA
669        Irwindale, CA
670    San Francisco, CA
671         New York, NY
Name: Location, Length: 672, dtype: object

In [169]:
# Split 'Location' into 3 columns
df_split = df['Location'].str.split(', ', expand=True)

# Create 'job state' column based on the length condition for the second column
#df['job state'] = df_split.apply(lambda row: row[1] if len(str(row[1])) != 2 else row[2], axis=1)
df['job state'] = df_split.apply(lambda row: row[2] if len(str(row[1])) != 2 else row[1], axis=1)


# Display the DataFrame with the new column
df['job state'].unique()

array(['NY', 'VA', 'MA', 'CA', 'IL', 'MO', 'WA', 'DC', None, 'TN', 'TX',
       'PA', 'AZ', 'WI', 'GA', 'FL', 'NE', 'KS', 'NH', 'NJ', 'LA', 'OH',
       'IN', 'MD', 'CO', 'UT', 'OR', 'MI', 'SC', 'MS', 'AL', 'RI', 'IA',
       'MN', 'OK', 'CT', 'NC', 'DE', 'WV'], dtype=object)

In [166]:
df['Location'].str.split(', ', expand=True)

,0,1,2
0,New York,NY,None
1,Chantilly,VA,None
2,Boston,MA,None
3,Newton,MA,None
4,New York,NY,None
...,...,...,...
667,Fort Lee,NJ,None
668,San Francisco,CA,None
669,Irwindale,CA,None
670,San Francisco,CA,None


In [175]:
df["Job Description"] = df["Job Description"].str.replace("\n", ' ')
df["Job Description"]

0      Description  The Senior Data Scientist is resp...
1      Secure our Nation, Ignite your Future  Join th...
2      Overview   Analysis Group is one of the larges...
3      JOB DESCRIPTION:  Do you have a passion for Da...
4      Data Scientist Affinity Solutions / Marketing ...
                             ...                        
667    Summary  We’re looking for a data scientist to...
668    Job Description Become a thought leader within...
669    Join a thriving company that is changing the w...
670    100 Remote Opportunity As an AINLP Data Scient...
671    Description  The Data Scientist will be part o...
Name: Job Description, Length: 672, dtype: object